In [ ]:
# Install required packages
!pip install numpy==1.26 scipy==1.14 meshio==5.3.5 libigl==v2.5.1 polyscope==2.2.1 ilupp==1.0.2 ipctk==1.2.0 networkx==3.3 noise pyvista plotly ipywidgets trame tetgen numpy-stl trimesh widgetsnbextension matplotlib tqdm


In [2]:
#!rm -rf *

In [3]:
import os
import zipfile
from concurrent.futures import ThreadPoolExecutor, as_completed

import meshio
import numpy as np
import tetgen as tg
import trimesh
from noise import pnoise3
from PIL import Image
from tqdm import tqdm

In [4]:
class HeightmapMeshGenerator:
    def __init__(self, size_x=100, size_y=100, amplitude=1.0, noise_function=None, output_folder='output', **noise_params):
        """
        Initialize the HeightmapMeshGenerator with given parameters.

        Parameters:
        - size_x (int): Number of points along the X-axis.
        - size_y (int): Number of points along the Y-axis.
        - amplitude (float): Scaling factor for the heightmap.
        - noise_function (callable): Function to generate noise.
        - output_folder (str): Directory to save all outputs.
        - noise_params (dict): Additional parameters for the noise function.
        """
        self.size_x = size_x
        self.size_y = size_y
        self.amplitude = amplitude
        self.noise_function = noise_function if noise_function is not None else self.fbm_noise
        self.noise_params = noise_params

        self.heightmap = None
        self.normals = None
        self.mesh_vertices = None
        self.mesh_faces = None
        self.tetrahedral_mesh = None

        # Attributes for normal map
        self.normal_map_scale = 1  # Default scale factor
        self.normal_map_resolution = (512, 512)  # Default resolution (width, height)

        # Attributes for displacement map
        self.displacement_map_scale = 1  # Default scale factor
        self.displacement_map_resolution = (512, 512)  # Default resolution (width, height)

        # Output folder setup
        self.output_folder = output_folder
        os.makedirs(self.output_folder, exist_ok=True)

    # -------------------------------
    # Noise Functions
    # -------------------------------
    def fbm_noise(self, x, y, z=0, octaves=4, persistence=0.5, lacunarity=2.0):
        """Fractal Brownian Motion noise."""
        value = 0.0
        amp = 1.0
        freq = 1.0
        for _ in range(octaves):
            try:
                noise_val = pnoise3(x * freq, y * freq, z * freq)
                value += amp * noise_val
            except Exception as e:
                print(f"Error generating fbm_noise at ({x}, {y}, {z}): {e}")
                value += 0  # Assign a default value or handle as needed
            amp *= persistence
            freq *= lacunarity
        return value

    def perlin_noise(self, x, y, z=0, scale=1.0):
      """Simple Perlin noise with an optional scale."""
      try:
          # Apply scaling directly to the coordinates
          return pnoise3(x * scale, y * scale, z * scale)
      except Exception as e:
          print(f"Error generating perlin_noise at ({x}, {y}, {z}): {e}")
          return 0  # Assign a default value or handle as needed

    def square_wave(self, x, y, z=0, frequency=10.0, noise_amplitude=1.0):
        """Square wave noise."""
        try:
            return noise_amplitude * np.sign(np.sin(frequency * x))
        except Exception as e:
            print(f"Error generating square_wave at ({x}, {y}, {z}): {e}")
            return 0

    def sine_wave(self, x, y, z=0, frequency=10.0, noise_amplitude=1.0):
        """Sine wave noise."""
        try:
            return noise_amplitude * np.sin(frequency * x)
        except Exception as e:
            print(f"Error generating sine_wave at ({x}, {y}, {z}): {e}")
            return 0

    def beckmann_noise(self, x, y, alpha=0.5):
        """Beckmann microfacet distribution."""
        try:
            tan_theta_h = np.sqrt(x**2 + y**2) / alpha
            cos_theta_h = 1 / np.sqrt(1 + tan_theta_h**2)
            D = np.exp(- (tan_theta_h**2)) / (np.pi * alpha**2 * cos_theta_h**4)
            return D
        except Exception as e:
            print(f"Error generating beckmann_noise at ({x}, {y}): {e}")
            return 0

    def ggx_noise(self, x, y, alpha=0.5):
        """GGX (Trowbridge-Reitz) microfacet distribution."""
        try:
            tan2_theta_h = (x**2 + y**2) / alpha**2
            cos_theta_h = 1 / np.sqrt(1 + tan2_theta_h)
            cos2_theta_h = cos_theta_h**2
            D = alpha**2 / (np.pi * ((cos2_theta_h * (alpha**2 - 1) + 1)**2))
            return D
        except Exception as e:
            print(f"Error generating ggx_noise at ({x}, {y}): {e}")
            return 0

    def blinn_noise(self, x, y, n=20):
        """Blinn-Phong microfacet distribution."""
        try:
            tan_theta_h = np.sqrt(x**2 + y**2)
            cos_theta_h = 1 / np.sqrt(1 + tan_theta_h**2)
            D = (n + 2) / (2 * np.pi) * cos_theta_h**n
            return D
        except Exception as e:
            print(f"Error generating blinn_noise at ({x}, {y}): {e}")
            return 0

    # -------------------------------
    # Heightmap Generation
    # -------------------------------
    def generate_heightmap(self):
        """Generate the heightmap using the selected noise function."""
        size_x, size_y = self.size_x, self.size_y
        x = np.linspace(-1, 1, size_x)
        y = np.linspace(-1, 1, size_y)
        heightmap = np.zeros((size_x, size_y))

        # Loop through each point and call the noise function with scalar inputs
        for i in range(size_x):
            for j in range(size_y):
                xi = x[i]
                yj = y[j]
                try:
                    heightmap[i, j] = self.noise_function(xi, yj, **self.noise_params)
                except Exception as e:
                    print(f"Error generating noise at ({xi}, {yj}): {e}")
                    heightmap[i, j] = 0  # Assign a default value or handle as needed

        # Normalize and scale the heightmap
        heightmap = self.normalize_heightmap(heightmap) * self.amplitude
        self.heightmap = heightmap

        # Compute normals after generating the heightmap
        self.compute_normals()

    def normalize_heightmap(self, heightmap):
        """Normalize the heightmap to range [0, 1]."""
        min_val = np.min(heightmap)
        max_val = np.max(heightmap)
        if max_val - min_val == 0:
            return np.zeros_like(heightmap)
        return (heightmap - min_val) / (max_val - min_val)

    # -------------------------------
    # Compute Normals
    # -------------------------------
    def compute_normals(self):
        """Compute the normals for the heightmap mesh."""
        dzdx = np.gradient(self.heightmap, axis=0)
        dzdy = np.gradient(self.heightmap, axis=1)
        normals = np.dstack((-dzdx, -dzdy, np.ones_like(self.heightmap)))
        norm = np.linalg.norm(normals, axis=2, keepdims=True)
        norm[norm == 0] = 1  # Prevent division by zero
        self.normals = normals / norm

    # -------------------------------
    # Filename Generator
    # -------------------------------
    def generate_filename(self, file_type):
        """Generate filename based on parameters."""
        noise_name = self.noise_function.__name__.replace("_noise", "").replace("_", "").capitalize()
        base_name = f"heightmap_{noise_name}_S{self.size_x}x{self.size_y}_A{self.amplitude:.2f}"

        if self.noise_function.__name__ in ['fbm_noise', 'perlin_noise']:
            base_name += f"_O{self.noise_params.get('octaves', 4)}P{self.noise_params.get('persistence', 0.5):.2f}L{self.noise_params.get('lacunarity', 2.0):.2f}"
        elif self.noise_function.__name__ in ['beckmann_noise', 'ggx_noise']:
            base_name += f"_A{self.noise_params.get('alpha', 0.5):.2f}"
        elif self.noise_function.__name__ == 'blinn_noise':
            base_name += f"_N{self.noise_params.get('n', 20)}"
        elif self.noise_function.__name__ in ['sine_wave', 'square_wave']:
            base_name += f"_F{self.noise_params.get('frequency', 10.0):.1f}A{self.noise_params.get('noise_amplitude', 1.0):.1f}"

        if file_type == "stl":
            return f"{base_name}.stl"
        elif file_type == "msh":
            return f"{base_name}.msh"
        elif file_type == "normal":
            return f"{base_name}_normal.png"
        elif file_type == "displacement":
            return f"{base_name}_displacement.png"
        return "output_file"

    # -------------------------------
    # Save Normal Map as PNG
    # -------------------------------
    def save_normal_map_as_png(self, filename=None):
        """Save the normal map as a detailed PNG image with adjustable resolution."""
        if self.normals is None:
            raise ValueError("Normals have not been computed yet.")

        if filename is None:
            filename = self.generate_filename("normal")

        # Map normals from [-1, 1] to [0, 255] for RGB representation
        normals_normalized = (self.normals + 1.0) / 2.0  # Scale to [0,1]
        normals_image = (normals_normalized * 255).astype(np.uint8)

        # Create the image using PIL
        img = Image.fromarray(normals_image, 'RGB')

        # Adjust resolution based on normal_map_resolution
        target_resolution = self.normal_map_resolution
        img = img.resize(target_resolution, Image.NEAREST)  # Use NEAREST to preserve normal data
        print(f"Normal map resized to {target_resolution} pixels.")

        # Apply scaling if scale factor is greater than 1
        if self.normal_map_scale != 1:
            new_size = (img.width * self.normal_map_scale, img.height * self.normal_map_scale)
            img = img.resize(new_size, Image.NEAREST)  # Use NEAREST to preserve normal data
            print(f"Normal map upscaled to {new_size} pixels.")

        # Save the image
        img.save(os.path.join(self.output_folder, filename))
        print(f"Normal map saved to {os.path.join(self.output_folder, filename)}")

    # -------------------------------
    # Save Displacement Map as PNG
    # -------------------------------
    def save_displacement_map_as_png(self, filename=None):
        """Save the displacement map as a grayscale PNG image with adjustable resolution."""
        if self.heightmap is None:
            raise ValueError("Heightmap has not been generated yet.")

        if filename is None:
            filename = self.generate_filename("displacement")

        # Normalize heightmap to [0, 255] for grayscale representation
        displacement_normalized = (self.heightmap * 255).astype(np.uint8)
        displacement_image = Image.fromarray(displacement_normalized, 'L')  # 'L' mode for grayscale

        # Adjust resolution based on displacement_map_resolution
        target_resolution = self.displacement_map_resolution
        displacement_image = displacement_image.resize(target_resolution, Image.NEAREST)  # Use NEAREST to preserve data
        print(f"Displacement map resized to {target_resolution} pixels.")

        # Apply scaling if scale factor is greater than 1
        if self.displacement_map_scale != 1:
            new_size = (displacement_image.width * self.displacement_map_scale, displacement_image.height * self.displacement_map_scale)
            displacement_image = displacement_image.resize(new_size, Image.NEAREST)  # Use NEAREST to preserve data
            print(f"Displacement map upscaled to {new_size} pixels.")

        # Save the image
        displacement_image.save(os.path.join(self.output_folder, filename))
        print(f"Displacement map saved to {os.path.join(self.output_folder, filename)}")

    # -------------------------------
    # Mesh Generation and Processing
    # -------------------------------
    def heightmap_to_closed_mesh(self, bottom_height=-2.0):
        """Convert the heightmap to a closed 3D mesh with adjustable bottom height."""
        size_x, size_y = self.heightmap.shape

        # Create grid of indices
        i = np.arange(size_x)
        j = np.arange(size_y)
        ii, jj = np.meshgrid(i, j, indexing='ij')

        # Flatten the grid indices
        ii_flat = ii.flatten()
        jj_flat = jj.flatten()
        height_flat = self.heightmap.flatten()

        # Generate top and bottom vertices
        top_vertices = np.column_stack((ii_flat, jj_flat, height_flat))
        bottom_vertices = np.column_stack((ii_flat, jj_flat, np.full_like(height_flat, bottom_height)))

        # Combine vertices
        vertices = np.vstack((top_vertices, bottom_vertices))

        # Index offset for bottom vertices
        offset = size_x * size_y

        # Create vertex indices grid
        vertex_indices = np.arange(size_x * size_y).reshape((size_x, size_y))
        vertex_indices_bottom = vertex_indices + offset

        # Indices for cells (quads)
        v0 = vertex_indices[:-1, :-1]
        v1 = vertex_indices[1:, :-1]
        v2 = vertex_indices[:-1, 1:]
        v3 = vertex_indices[1:, 1:]

        # Flatten the indices
        v0_flat = v0.flatten()
        v1_flat = v1.flatten()
        v2_flat = v2.flatten()
        v3_flat = v3.flatten()

        # Create faces for the top surface
        faces_top = np.vstack([
            np.column_stack([v0_flat, v1_flat, v2_flat]),
            np.column_stack([v1_flat, v3_flat, v2_flat])
        ])

        # Bottom surface faces (reverse the order to flip normals)
        v0b = vertex_indices_bottom[:-1, :-1]
        v1b = vertex_indices_bottom[1:, :-1]
        v2b = vertex_indices_bottom[:-1, 1:]
        v3b = vertex_indices_bottom[1:, 1:]

        v0b_flat = v0b.flatten()
        v1b_flat = v1b.flatten()
        v2b_flat = v2b.flatten()
        v3b_flat = v3b.flatten()

        faces_bottom = np.vstack([
            np.column_stack([v0b_flat, v2b_flat, v1b_flat]),
            np.column_stack([v1b_flat, v2b_flat, v3b_flat])
        ])

        # Side faces
        # Left side (j=0)
        v_top_left = vertex_indices[:, 0]
        v_bot_left = vertex_indices_bottom[:, 0]

        faces_left = np.vstack([
            np.column_stack([v_top_left[:-1], v_bot_left[:-1], v_bot_left[1:]]),
            np.column_stack([v_top_left[:-1], v_bot_left[1:], v_top_left[1:]])
        ])

        # Right side (j=size_y-1)
        v_top_right = vertex_indices[:, -1]
        v_bot_right = vertex_indices_bottom[:, -1]

        faces_right = np.vstack([
            np.column_stack([v_top_right[:-1], v_bot_right[1:], v_bot_right[:-1]]),
            np.column_stack([v_top_right[:-1], v_top_right[1:], v_bot_right[1:]])
        ])

        # Front side (i=0)
        v_top_front = vertex_indices[0, :]
        v_bot_front = vertex_indices_bottom[0, :]

        faces_front = np.vstack([
            np.column_stack([v_top_front[:-1], v_bot_front[:-1], v_bot_front[1:]]),
            np.column_stack([v_top_front[:-1], v_bot_front[1:], v_top_front[1:]])
        ])

        # Back side (i=size_x-1)
        v_top_back = vertex_indices[-1, :]
        v_bot_back = vertex_indices_bottom[-1, :]

        faces_back = np.vstack([
            np.column_stack([v_top_back[:-1], v_bot_back[1:], v_bot_back[:-1]]),
            np.column_stack([v_top_back[:-1], v_top_back[1:], v_bot_back[1:]])
        ])

        # Combine all faces
        faces = np.vstack((faces_top, faces_bottom, faces_left, faces_right, faces_front, faces_back))

        self.mesh_vertices = vertices
        self.mesh_faces = faces

    def repair_mesh(self):
        """Repair the mesh if it's not watertight."""
        mesh = trimesh.Trimesh(vertices=self.mesh_vertices, faces=self.mesh_faces)
        if not mesh.is_watertight:
            print("Mesh is not watertight. Attempting to repair...")
            try:
                # Attempt to repair the mesh
                trimesh.repair.fill_holes(mesh)
                trimesh.repair.fix_normals(mesh)
                trimesh.repair.fix_inversion(mesh)
                # Remove duplicate faces or vertices
                mesh.remove_duplicate_faces()
                mesh.remove_duplicate_vertices()
                mesh.remove_unreferenced_vertices()
                # Check again if the mesh is watertight
                if not mesh.is_watertight:
                    print("Warning: Mesh is still not watertight after initial repair. Attempting further repairs...")
                    trimesh.repair.fill_holes(mesh)
                    trimesh.repair.fix_normals(mesh)
                    trimesh.repair.fix_inversion(mesh)
                    mesh.remove_duplicate_faces()
                    mesh.remove_duplicate_vertices()
                    mesh.remove_unreferenced_vertices()
                    if not mesh.is_watertight:
                        raise RuntimeError("Mesh is not watertight after repair.")
                print("Mesh repaired successfully.")
                self.mesh_vertices = mesh.vertices
                self.mesh_faces = mesh.faces
            except Exception as e:
                print(f"Error during mesh repair: {e}")
                raise
        else:
            print("Mesh is watertight.")

    def simplify_mesh(self, target_face_count):
        """Simplify the mesh to a target number of faces."""
        mesh = trimesh.Trimesh(vertices=self.mesh_vertices, faces=self.mesh_faces)
        try:
            simplified_mesh = mesh.simplify_quadratic_decimation(target_face_count)
            self.mesh_vertices = simplified_mesh.vertices
            self.mesh_faces = simplified_mesh.faces
            print(f"Simplified mesh to {len(self.mesh_faces)} faces.")
        except Exception as e:
            print(f"Error during mesh simplification: {e}")

    def save_mesh_to_stl(self, filename=None):
        """Save the mesh to an STL file."""
        if filename is None:
            filename = self.generate_filename("stl")
        meshio_mesh = meshio.Mesh(points=self.mesh_vertices, cells=[("triangle", self.mesh_faces)])
        try:
            meshio.write(os.path.join(self.output_folder, filename), meshio_mesh)
            print(f"Mesh saved to {os.path.join(self.output_folder, filename)}")
        except Exception as e:
            print(f"Error saving STL file: {e}")

    def tetrahedralize(self, input_file='heightmap.stl', output_file='heightmap_tet.msh'):
        """Perform tetrahedralization on an STL file and save the result."""
        # Construct full paths for input and output files
        input_file_path = os.path.join(self.output_folder, input_file)
        output_file_path = os.path.join(self.output_folder, output_file)

        if not os.path.isfile(input_file_path):
            raise FileNotFoundError(f"File {input_file_path} not found.")

        try:
            # Read the STL file
            imesh = meshio.read(input_file_path)
            V = imesh.points
            F = imesh.cells_dict.get("triangle", [])

            if len(F) == 0:
                raise ValueError(f"No triangular faces found in {input_file_path}.")

            # Create TetGen mesher object
            mesher = tg.TetGen(V, F)

            try:
                # Perform tetrahedralization
                tet_result = mesher.tetrahedralize(order=1, mindihedral=5.0, minratio=1.0)
                if isinstance(tet_result, tuple):
                    Vtg, Ctg = tet_result
                else:
                    Vtg = tet_result
                    Ctg = mesher.grid.tetrahedrons
            except RuntimeError as e:
                raise RuntimeError("Failed to tetrahedralize. May need to repair surface by making it manifold.") from e

             # Create and save the tetrahedral mesh
            omesh = meshio.Mesh(Vtg, [("tetra", Ctg)])
            os.path.join(self.output_folder, output_file)
            meshio.write(os.path.join(self.output_folder, output_file), omesh)
            self.tetrahedral_mesh = omesh
            print(f"Tetrahedral mesh saved to {output_file}")

        except Exception as e:
            print(f"Error during tetrahedralization: {e}")
            raise

    # -------------------------------
    # Saving All Outputs
    # -------------------------------
    def save_all(self):
        """Generate heightmap, normal map, displacement map, mesh, and tetrahedral mesh, then save all to the output folder."""
        try:
            # Generate heightmap
            print("Generating heightmap...")
            self.generate_heightmap()

            # Save normal map
            print("Saving normal map...")
            normal_map_filename = self.generate_filename("normal")
            self.save_normal_map_as_png(normal_map_filename)

            # Save displacement map
            print("Saving displacement map...")
            displacement_map_filename = self.generate_filename("displacement")
            self.save_displacement_map_as_png(displacement_map_filename)

            # Generate and repair mesh
            print("Generating mesh...")
            self.heightmap_to_closed_mesh()
            self.repair_mesh()

            # Save mesh
            print("Saving mesh to STL...")
            stl_filename = self.generate_filename("stl")
            self.save_mesh_to_stl(stl_filename)

            # Tetrahedralize and save mesh
            print("Performing tetrahedralization...")
            msh_filename = self.generate_filename("msh")
            self.tetrahedralize(stl_filename, msh_filename)

            print("All files have been generated and saved successfully.")
        except Exception as e:
            print(f"An error occurred during the save_all process: {e}")


In [5]:
def zip_output(output_folder):
    zip_filename = f"{output_folder}.zip"
    zip_filepath = os.path.join(output_folder, zip_filename)

    with zipfile.ZipFile(zip_filepath, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=output_folder)  # Relative path for zip file structure
                zipf.write(file_path, arcname)

    print(f"Zipped folder saved to {zip_filepath}")
    return zip_filepath

In [6]:
# Instantiate a temporary generator to access noise functions
temp_generator = HeightmapMeshGenerator()

# Define the list of noise configurations
# Define ranges for each parameter
octave_values = [4, 6, 8]
persistence_values = [0.4, 0.5, 0.6]
lacunarity_values = [2.0, 2.5, 3.0]
frequency_values = [5.0, 10.0, 20.0]
amplitude_values = [1.0, 1.5, 2.0]
alpha_values = [0.3, 0.5, 0.7]
blinn_n_values = [10, 20, 50]

# List to store generated noise configurations
generated_noise_configs = []

# Generate FBM noise configurations
for octaves in octave_values:
    for persistence in persistence_values:
        for lacunarity in lacunarity_values:
            generated_noise_configs.append({
                'name': f'fbm_noise_o{octaves}_p{persistence}_l{lacunarity}',
                'function': temp_generator.fbm_noise,
                'params': {
                    'octaves': octaves,
                    'persistence': persistence,
                    'lacunarity': lacunarity
                }
            })

# Generate Perlin noise configurations
generated_noise_configs.append({
    'name': 'perlin_noise_default',
    'function': temp_generator.perlin_noise,
    'params': {}
})

generated_noise_configs.append({
    'name': 'perlin_noise_scaled',
    'function': temp_generator.perlin_noise,
    'params': {
        'scale': 0.5  # Custom scale factor for stretching noise
    }
})

# Generate Sine wave noise configurations
for frequency in frequency_values:
    for amplitude in amplitude_values:
        generated_noise_configs.append({
            'name': f'sine_wave_f{frequency}_a{amplitude}',
            'function': temp_generator.sine_wave,
            'params': {
                'frequency': frequency,
                'noise_amplitude': amplitude
            }
        })

# Generate Square wave noise configurations
for frequency in frequency_values:
    for amplitude in amplitude_values:
        generated_noise_configs.append({
            'name': f'square_wave_f{frequency}_a{amplitude}',
            'function': temp_generator.square_wave,
            'params': {
                'frequency': frequency,
                'noise_amplitude': amplitude
            }
        })

# Generate Beckmann noise configurations
for alpha in alpha_values:
    generated_noise_configs.append({
        'name': f'beckmann_noise_alpha{alpha}',
        'function': temp_generator.beckmann_noise,
        'params': {
            'alpha': alpha
        }
    })

# Generate GGX noise configurations
for alpha in alpha_values:
    generated_noise_configs.append({
        'name': f'ggx_noise_alpha{alpha}',
        'function': temp_generator.ggx_noise,
        'params': {
            'alpha': alpha
        }
    })

# Generate Blinn-Phong noise configurations
for n in blinn_n_values:
    generated_noise_configs.append({
        'name': f'blinn_noise_n{n}',
        'function': temp_generator.blinn_noise,
        'params': {
            'n': n
        }
    })

# Define the amplitudes you want to iterate over
amplitude_values = [1.0, 1.5, 2.0]

In [ ]:
# Function to handle generation and saving of heightmaps and other files
def generate_mesh(amplitude, noise_config, output_folder):
    stl_filename = f"heightmap_{noise_config['name']}_S100x100_A{amplitude:.2f}.stl"
    stl_filepath = os.path.join(output_folder, stl_filename)

    if os.path.exists(stl_filepath):
        print(f"Skipping existing file: {stl_filepath}")
        return None  # Skip existing files

    try:
        generator = HeightmapMeshGenerator(
            size_x=100,
            size_y=100,
            amplitude=amplitude,
            noise_function=noise_config['function'],
            output_folder=output_folder,
            **noise_config['params']
        )

        # Generate and save all outputs
        generator.save_all()

        # Zip the output folder after generation
        zip_filepath = zip_output(output_folder)
        return zip_filepath

    except Exception as e:
        print(f"Failed to generate for amplitude={amplitude}, noise={noise_config['name']}: {e}")
        return None

# List of results (zip files)
results = []

# Using ThreadPoolExecutor for multithreading
with ThreadPoolExecutor() as executor:
    futures = []

    # Loop through amplitudes and noise configurations
    for amplitude in tqdm(amplitude_values, desc="Processing Amplitudes"):
        for noise_config in tqdm(generated_noise_configs, desc="Processing Noise Configurations", leave=False):
            output_folder = f'output_amp{amplitude}_noise{noise_config["name"]}'

            # Submit each task to the ThreadPoolExecutor
            futures.append(executor.submit(generate_mesh, amplitude, noise_config, output_folder))

    # Collect results as they are completed
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            results.append(result)

# Optionally, zip all results together into one large archive
final_zip = "all_results.zip"
with zipfile.ZipFile(final_zip, 'w', zipfile.ZIP_DEFLATED) as final_zipf:
    for zip_path in results:
        arcname = os.path.basename(zip_path)  # Use the zip file's name in the archive
        final_zipf.write(zip_path, arcname)

print(f"All results zipped into {final_zip}")

In [ ]:
!zip -r heightmap.zip *